# Ideal QKD Implementation with EVE

This implementation will make the assumption that Eve is not present. The cells below contain the photon and qubit modules. Normally we would save these as python files and import them using statements, similar to how we have used statements like "import random" or "import math". Due to the constraints of using Colab, we are going to store them in hidden cells. Take a look at the classes. What are the methods? What are the members? Once you are done, put "#@title" at the top of each cell and press play. Click on the whitespace to the right. If you did it right, the cell should collapse and you should only see "SHOW CODE" in the cell.





In [1]:
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    

    def measureDA(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

# Alice creates a message

The length of the message will determine how long the key, and what number of qubits they will need to use. 


In [2]:
alice_message = "hello"   #put any message you like here, try to keep it small though!
#You will need 8 bits per character.

#this parameter will become more important in the next phase of your project. It gives you 
# some leeway in case some of the key gets intercepted. You can ignore this for now.


safetyBuffer = 1000


# Alice generates the raw key

In [3]:


n = 8*len(alice_message) + safetyBuffer # number of qubits. We multiply by 8 because of ASCII

# Alice --------------------------------------------


rawKeyAlice = ""
for i in range(n): # Iterate over the number of qubits.
    # Append a random character ('0' or '1') to the end.
    rawKeyAlice += random.choice(['0','1'])
print("keyAlice    = " + rawKeyAlice)


keyAlice    = 11111010111111001110100010100110001110100000101101000011101001001100011011011101001110000000100110111110011000101001100111000001011100110110010101111101000000101011101000000011110011110000001110100100111000110100100110111111011111000101011111001110101111100100010011011000111100010111111110111000001000111111011101111101001000000000110011000101011011111101100011011110111011010001110101100011100011001111000101100010001010111011111011011000001010110110000110011111001100100010000001001000111110001000101001010101011100001100110000001011001000010010001100100010010110100000010001001001001101111101010111001010011010100101011010001011001001001001101010001110011011001010100100101100000010100011110110100011100011111101000010001011010000010000011000011001001010101111101000111000011001010000000111101010011110010100011010010100011011000111111111011100011100010010001001111111011001010001001111100001110011001010101101000110011101101101001001110001101010001101101111001101110101011011011101

# Alice chooses the encoding basis for each key bit.
 This should be a randomly chosen string of '+'s and 'x's with '+'=H/V, 'x'=D/A. You do not need to create a function for this unless you want to. A simple "for loop" will be fine.




In [4]:
basisAlice = ""
# TODO: Put your code here.
for i in range(n):
    basisAlice += random.choice(['+','x'])
print("basisAlice  = " + basisAlice)


# Your output should just be a bunch of  '+' and  'x' s, with no spaces in between.

basisAlice  = ++x+x++x+++xx+++++x+++++++xxx+xxx+x+x++x+xxx+xxx+++xx+++xx+x++x+++x+x+xx+xx+x+xxxx+x++x++++xxxxx+++++xx+xx++xx++xx+xx+x++xx++++x+xxx+xx++++xx+++++xxxxxx+x+++xx+xx++x+xxx++x+x+x++++xx++xx+xxxx+xxx+xxxx+x++x++++++xx++xx+xx+x+x++xxx+x+xx+xx+x+xxx+++xx+x+x++xxxx+xxxx+x+xxx+x+++x+x++xx++x++x++xx++xxx+x++++x+x+xxxx+xxxx+xxx++x+x++x++xx+xx++++xxx+++x+xxxx+x+x+x++xxxxxx++++xx+xxxx+xxx+xxx+xx++x+++++++xx+xx+++xx+xx+x++x++xxxx+xxxx+x+x+x++x++++xx++x+++x++x++xx++xx+x++xxx+x+x+xxx+xx+x++xxx+x+++x+xxx++x+xxxx++xxxx++x+xx++x+xx++xxx+x+++++xxxxx+++x+++xxx++x+xxx+x+x+++++x++x++xxxxx+++xxx+xxx+xxxx+++x+++xx+xxxxx++xx+xxx+x++x+xxxx++++++x++xxx+x++xxx+x+xx+x+x++++x+xx+++xxx+xx+++++xx+x++xx++x+xxxx++xx++xxx+x+x+++++x+x+x+x+x++xxxx+x+x+xx++x+xx+x+++++x+++++x+x+++xxxx+xx+x+++xx+x+++++xxxxx+x++x++++x+xx++++++x+++x++xxx++x++x+x+xxx+xx++x++x++x+x+xx+++xx++x+x++++xxxxxx+xx++x+++x+xx+x+x+x+xx++++x+++++++x++++xxxxx+++xx++x+++xxxxxxxx+++xxx++x+x++xx+++xxxxxx+x++xx+++++++xxx+xx+x+xx+x++++xxxxx++xx+++x

# Alice selects a qubit state according to the key and basis.

Write a function that takes basisAlice ( string of +, x) and rawKeyAlice (a string of 0's and 1's) as arguments and returns photonAlice, a string of polarizations  'H', 'V', 'D',  or 'A'.

In [5]:


def photonAlice_generator( your_basisAlice,your_rawKeyAlice ):
# TODO: Put your code here.
    your_photonAlice = ""
    for i in range( len (your_basisAlice)):
        if your_basisAlice[i]=='+':
            if your_rawKeyAlice[i]=='0':
                your_photonAlice += 'H'
            elif your_rawKeyAlice[i]=='1':
                your_photonAlice += 'V'
        elif your_basisAlice[i]=='x':
            if your_rawKeyAlice[i]=='0':
                your_photonAlice += 'D'
            elif your_rawKeyAlice[i]=='1':
                your_photonAlice += 'A'
    #print("photonAlice  = " + photonAlice)
    return your_photonAlice





# Sanity check: photonAlice_generator()


In [6]:
test_basis = '+x+'

test_key = '010'

test_photon =  photonAlice_generator(test_basis, test_key)

print(test_photon)

# you should see: HAH

HAH


In [7]:
#create qubitAlice in order to be passed into the next part of the protocol

photonAlice = photonAlice_generator(basisAlice, rawKeyAlice)

print(photonAlice) #just to check!


VVAVAHVDVVVAAVHHVVAHVHHHVHADDVADDHAVAHVDHDDDVDAAHVHDDHVVADVDHVDHVVDHDVADVADVAVDADDVAVHDHHHHDADDAVHVVVAAHDAVHDDVHADHAAHDVVADHHHHAHAAAHDAVHVVDDVHVHVAAAADAHDHHHDAHADVVAHADDHHDHDVAVVHHAAVVDDHDDDAVADAHDADDVAVHDHVVHVHDAHHAAHAAVAVAHVAAAVDHDAHADVAVAADHVVADVDVAVVADDAHDDADHAVDAAHDHVVAVDHHADVVAVVAVVDAVVDDDHDVHHHAVAVAADAVADAAVAADVHDVDHHDHHDDHAAHHVVDDDVHVDVADAAVAVAHAVHDDAADAVVVHAAVDAADVDDDVAADVDAVHDHVVVHHHAAHDAVVVDDHADVAHHDVHDDADVDAAAHAVAVAHVAHVVHDDHHAHVHAVHAVHDDHVADHAVVAADHAVDHADDHADHDHHDADHAHHHAVAAAHHDVDDDAHVDDADVHAHADVVAHDDHVADDVAHHHHHDADAAHHVDHHHADDVHDHAADHAHDHVHHVDVVDVHDDDDDVHHDADHADDVDDAAHVVAVVHADVDAAADHVDAHDAAHAHVDHADADVVHVHHDVHAADHAHHADDVDHAAHAHAHHHVAVDDVVHAADHADVHVHHADHAHVADHHDHDADAHHDAVVADAVDVDHHVVVDHDVAVAVAHVDDDDVDHDVDAVHAHDDHDVHHHHDVVHHHDVAHHVDDADVDAHAVVVADVDHHVVVDDDDAVDHVDVHHHDHDDVVVVHVDVHHAVVADDVHAHHDVAHADDVDAHHDVVDVVDHDVAAVVVAAVHAVAHHHVAADDDAHDAHHDVHHAVAAVAVDVAHDAHVHHDVHHVVVVAHHHHAAADDVVHDAHVDVHVADADDDAAHHVAADVVDVAHVDDVHHAAADDDVAHVDAHHHVVHVADAVAAHDVADVAVHVHADAADVVDAVVHAHVHADAHDDHAHDV

# Alice prepares and sends each qubit.
Create a function that includes the photonArray initialization listed below. This function shoud take in photonAlice as an argument, and then use the correct method of the Photon class (.prepareH(), .prepareV(), .prepareD(), .prepareA(), etc.)

In [8]:
average_photon_number = 1 # playwith this : insert [description of parameter]

def photonArray_generator(your_photonAlice,average_photon_number):

    your_photonArray = [Photon() for i in range(len(your_photonAlice))] # Initializes an array of instances of the Photon class

    # TODO: Put your code here.
    for i in range(len(your_photonAlice)):
        if   your_photonAlice[i]=='H': your_photonArray[i].prepareH(average_photon_number)
        elif your_photonAlice[i]=='V': your_photonArray[i].prepareV(average_photon_number)
        elif your_photonAlice[i]=='D': your_photonArray[i].prepareD(average_photon_number)
        elif your_photonAlice[i]=='A': your_photonArray[i].prepareA(average_photon_number)
    return your_photonArray

    

In [9]:
photonArray = photonArray_generator(photonAlice,1)

In [10]:
random.seed(123)

test_photonAlice  = 'HAH'

test_photonArray = photonArray_generator(test_photonAlice,1)

print(len(test_photonArray))

for i in range(len(test_photonArray)):
   print(test_photonArray[i].toString())

# you should see : 
#(1.2021142166123333+0.06900569632415206j)|H> + (-0.1992923612468027+0.13137399451330123j)|V>
#(0.8205500555172319-0.08112282097832324j)|H> + (-1.1448454295134607-0.10132338482239818j)|V>
#(1.1764607134499159-0.23629937837582562j)|H> + (-0.23477638910021187+0.3846557133557717j)|V>


3
(1.2021142166123333+0.06900569632415206j)|H> + (-0.1992923612468027+0.13137399451330123j)|V>
(0.8205500555172319-0.08112282097832324j)|H> + (-1.1448454295134607-0.10132338482239818j)|V>
(1.1764607134499159-0.23629937837582562j)|H> + (-0.23477638910021187+0.3846557133557717j)|V>


# Eve   --------------------------------------------
 You should implement this section after completing Alice and Bob.Eve is allowed to do whatever she wants to the photonAlice array. She cannot, however, have knowledge of Alice's or Bob's choice of bases, nor Bob's measurement outcomes, until they are publicly announced.Eve selects a subsample of photons from Alice to measure.  "interceptIndex" should be a string of n characters. Use the convention '0'=ignored, '1'=intercepted. For ex : A string of only 0's would mean Eve ignores every photon (bad idea) and a string of 1's would mean she measures every photon (also a bad idea). It would be a very good idea (hint hint) to crreate a variable that counts the number of intercepted photons. 

In [11]:
interceptIndex = ""


# TODO: Put your code here.

# Some considerations : How are you going to choose which photons to sample? How can you do this 
# in a way that is repeatable with different values of "n ", or the total number of photons? How can you do this 
# in a way that makes it easy to vary the number of photons you sample? Feel free to experiment or 
#even write functions to create interceptIndex to your specifications

fracIntercepted = 0.1 # the probability Eve chooses to intercept a photon
numIntercepted = 0    # a running total of the number of interceptions
for i in range(n):
    if random.uniform(0,1) < fracIntercepted:
        interceptIndex += '1'
        numIntercepted += 1
    else:
        interceptIndex += '0'




# Sanity Check - interceptIndex

In [12]:

print(interceptIndex)

if len(interceptIndex)== n:
    print( "Length is correct")



#Should print : A string of 1's and 0's that is the same length as " n " or the number of photons


0101010001001000000000000000000000010000000010000101000000000000000100000000101010100000000000000001010000000000000100000000010000000000000001000000100000000000000000000000001000000000001000000000000100000000000000001000100000000000000001000000010000000010000010000000000001010100100000000001000000000010000000100000000010000001011000010010000000011000011000000000000000000010001000000001000000010000000000011000000000000000010000000000100000000001100000000001000000000000100000000011001000100000000000100000000000000000000000000000000000000000000010000000000000000000000000000000000000010000000000000011000000000001000000000000101001000100000000000000000000110000001000010000000000000000000000001001000000100000000000101000000000000000100001000010010010100000000000000000000000000000010000000001000000101001000000000000000000000110000000000000000000000100000000000001100000000000000000000010000001000000000000101000000000001000000000000000100000000000000000000000000001000000000000000000100000000000

# Eve chooses a basis to measure each intercepted photon.
basisEve should be a string of n characters.
 Use the convention '0'=H/V, '1'=D/A, ' '=not measured. Try writing a function that takes interceptIndex as an argument and output basisEve.

In [13]:


# TODO: Put your code here. basisEve should be a blank string ' ', if interceptIndex is a '0' and 
# a random choice of bases if interceptIndex is a '1'.

def basisEve_generator(your_interceptIndex):
    
    basisEve = ""

    for i in range(len(your_interceptIndex)):
        if your_interceptIndex[i] == '1':
            basisEve += random.choice(['0','1'])
        else:
            basisEve += ' '
    return basisEve

                

# Sanity check - basisEve
Only run this cell if you are testing, the random.seed() line can cause you some problems down the road.


In [14]:

test_interceptIndex = '101'
random.seed(123)


test_basisEve = basisEve_generator(test_interceptIndex)

print(test_basisEve)

#You should see: '0 1'


0 1


In [15]:
#Once you have passed the sanity check run this:

basisEve = basisEve_generator( interceptIndex)

# Eve performs a measurement on each photon.
The function outcomeEve_generator() should take in photonArray, basisEve, interceptIndex and prob_dark_count as arguments and return outcomeEve. outComeEve should be appended measurements in HV or DA using methods of the Photon class, with prob_dark_count passed in as arguments. The variable "outcomeEve" should be a string of n characters.Use the convention 'H','V','D','A'; or a  ' ' if the photon is not measured.

In [16]:

def outcomeEve_generator(your_photonArray, your_basisEve, your_interceptIndex, your_prob_dark_count):
# TODO: Put your code here.

    your_outcomeEve = ""

    for i in range(len(your_interceptIndex)):
        if your_interceptIndex[i] == '1':
            if your_basisEve[i]=='0':
                your_outcomeEve += your_photonArray[i].measureHV(your_prob_dark_count)
            elif your_basisEve[i]=='1':
                your_outcomeEve += your_photonArray[i].measureDA(your_prob_dark_count)
        else:
            your_outcomeEve += ' '
            
    return  your_outcomeEve


# Sanity check: outcomeEve_generator





In [17]:
random.seed(123)

test_prob_dark_count = 0.1
test_outcomeEve =  outcomeEve_generator(test_photonArray, test_basisEve, test_interceptIndex, test_prob_dark_count)

print(test_outcomeEve)

#You should see: 'N N'

N N


In [18]:
#Run this once you've passed the sanity check

prob_dark_count = 0.1

outcomeEve = outcomeEve_generator(photonArray, basisEve, interceptIndex, prob_dark_count)

# Bob   --------------------------------------------

Bob chooses a basis to measure each qubit. (This is similar to what Alice does.)

In [19]:
basisBob = ""
# TODO: Put your code here.
for i in range(n):
    basisBob += random.choice(['+','x']) # '+'=H/V, 'x'=D/A
print("basisBob    = " + basisBob)



basisBob    = ++xx++xxx++xx+x+x+x++++++x+xxx+++x+x++++++xxxx+xx++++++xxxxxx+++++x+x+xxxxxx+++x++xxx++++x++x+xxxxxxx+++++x++++x+++x+x+xxx++xxxx+xx+xxx+++xxx+xx++++x+xx+x+xxx+x+++++xxxx+++x+xxx+x+xx++x++x++++xxxx+x++xxxxxx+x+++xxx++xxx+xxx++x++x+x+x++xxxxxxxxx+xx+xx++++xxx+x++x+x+xx++x++xx++xx++x++x+x+++x+x++++x+++xx+++++x++x+x+xx+xxxx++++xx+++++x++x++xx++x+xxxx++++++xxx++xx+xx+x++xx+xxx+xxxx++xxx+++xxxxx++++x+x++xxx+xxxx++x+++xxx+xxx+++x+x+x+++++xxx+xxxx+++xx+x+xxxx++++xx++x+x++++++xxx+xx+++++xxxx+xx++++x+x+x++x+xx+++xx+xx+xxxxxxx+xx+++x+xxx++x+++++x+x+xx+++++++xxx+xx+xx+x+x+x++xxx++x++x++++x+x+++x++++xx++xx++x+x+x++x+++++++xx++++xxx++x++x+x+xxx+xx+++++xx+++x+++xx+x++x+xxxx++x+xxxx+++xx++x+xx+xxxx++x++xx+x+x++++xx+xxx+++xxx+x+xx+x++++x+x+x+xx+xx++x++xx+xxx++x+++xx+x++x++++++++++xxx+xx++x+xx++++x+x+x+xxxx+++xx++x++xxx+xx++x+xx+++++++xxx+++xxx++xxxxxxxx+++x+x+++x+++xxx+x+x+xxxx+++x+x+++x++xxxxx+++xx+++xxx+xx++x+xxxxx+++xx+x+xx+xxxxxxx++x+xx+xx+++xx+x++x+x+x+x+++++++++++x++xxx+x++x++x++++x

# Bob performs a measurement on each qubit.

In [20]:
prob_dark_count = 0.1

# Use the methods of the Qubit class to measure each qubit.
outcomeBob = ""
# TODO: Put your code here.
for i in range(n):
    if basisBob[i]=='+':
        outcomeBob += photonArray[i].measureHV(prob_dark_count)
    elif basisBob[i]=='x':
        outcomeBob += photonArray[i].measureDA(prob_dark_count)
print("outcomeBob  = " + outcomeBob)
# This should be a string of the characters 'H', 'V', 'D', 'A'.

outcomeBob  = NNANNNDNNNNANVNNDVAHNNHNVNMDDDNVNDHANNNNNMNDNDHNMNNNVHNNAANNNNNNVNNNDNDDNANNNVNANNNNNNHHHNHNAVNANNDNNVNHNVNHNNNNVVHNNDNANNNNANNNNAAMDNANNVMDDNNNHHNVNVNNNNHNADHNNMNHVANNDNHNNNNNDNDNAMVVDNNDNHNNNNNDNNVHAADNDNVNHVHDNAHMNNNHANNHHANNNNNHANNADNNDNANAVNNNDDVHNVNNDNMVNANNNDDVVNHHMNVNNDNNNNVNVAVNMNNNNNNNNNMNNNNNNNNAVNNNNNNDVADDNNNMHNDNHNNHNHNNNVNNNNAVDNANNNNNVNNNDVHNNNDAVNNHANNNMNNDDNDNNADNNNNNDMDNVHNNNHNNNANAVDNNNVNANNNDNNNDNDNVVNHMNNHNNNNNNAHDNNNNNNANNNVNDNNVNNNDNHNANNNVNNNVNNNNNDNNMHNNMNNHNNNNVVDVNMNNNNNDNHNNNNHNDVNNANNNNHDNVNNDNNNANNANHNMNDNAMNDVNNHVNMNNANNNHNNNAVNVNVNDNDNHNNNNNNNHANMVNNANNNVNANVNNHNNNMVANNNVNMNNNHNDMHVNDNDHVNNNAVNNDNANANNHNHNNAHNNNVHNDNVNNNAHNANNHVAHNNDNNNNNAHNANADNNDNNNVANNNNVNNNVVVNNMNAMNNMNNDNNDNNNNANNNNAHNVNNNNNNANNNHHNNVANNVNDHNNDNNANVNNNVNHVNVNMNDNNADNNDNDNHNNNNNDVMNNNNANVNANNVNHNDDDNAANNDVDNNVNVNHNNNNNNNNNAVHNDNANAANNNNNNNNNNNVNNNNNNAVNNAANDMMNNHANNNDNHNNNNNNHHNNNVNANNVDNNNDVAANANHHNNANANANNMDNNDADNMNNDANNANNNNNNNVNNHNNDNNNNNNVNHMNVNANNNDNNNHNNVVNNNNHN

In [21]:
# Bob infers the raw key.
rawKeyBob = ""
for i in range(n):
    if outcomeBob[i] == 'H' or outcomeBob[i] == 'D':
        rawKeyBob += '0'
    elif outcomeBob[i] == 'V' or outcomeBob[i] == 'A':
        rawKeyBob += '1'
    elif outcomeBob[i] == 'N' or outcomeBob[i] == 'M':
        rawKeyBob += '-'

print(rawKeyBob)

--1---0----1-1--0110--0-1--000-1-001-------0-00-----10--11------1---0-00-1---1-1------000-0-11-1--0--1-0-1-0----110--0-1----1----11-0-1--1-00---00-1-1----0-100----011--0-0-----0-0-1-110--0-0-----0--10110-0-1-0100-10----01--001-----01--10--0-1-11---0010-1--0--1-1---0011-00--1--0----1-111--------------------11------01100----0-0-0--0-0---1----110-1-----1---010---011--01------00-0--10-----0-0-10---0---1-110---1-1---0---0-0-11-0---0------100------1---1-0--1---0-0-1---1---1-----0---0-----0----1101-------0-0----0-01--1----00-1--0---1--1-0---0-1--01--01----1---0---11-1-1-0-0-0-------01--1--1---1-1-1--0----11---1-----0-0-01-0-001---11--0-1-1--0-0--10---10-0-1---10-1--0110--0-----10-1-10--0---11----1---111----1------0--0----1----10-1------1---00--11--1-00--0--1-1---1-01-1---0--10--0-0-0-----01-----1-1-1--1-0-000-11--010--1-1-0---------110-0-1-11-----------1------11--11-0----01---0-0------00---1-1--10---0111-1-00--1-1-1---0--010----01--1-------1--0--0------1-0--1-1---0---0--11----0--------10--0--

# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# -----------------------------------------------------------

In [22]:
# Alice and Bob extract their sifted keys.
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.
for i in range(n):
    if basisAlice[i] == basisBob[i]:
        siftedAlice += rawKeyAlice[i]
        siftedBob   += rawKeyBob[i]
    else:
        siftedAlice += ' '
        siftedBob   += ' '
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

siftedAlice = 111  0 0 11111 0 11010001  00        01 0 00 0 1 10  01 10 0 1 011000110 10  1 1   1 0 00 0 1 01       0   0  1   01     1 0   1011  01101 001  01  1 01000  0    11  10000    1 1 011110 00   1101  1   1  0 1 01001 0 1 1  1  0   11000 010 1 110 1 1  01 11100    1    0    0   10 0 01111  110  1     10   1 1 1    0 1  10   1 0 000  01 0 1100 1 10 10  1 1  1 0 01 011 10111011  0001 10   1 0   10001        0 101  0 1 00 0 0         01 0    0  10101 011 00 1  01 1 1   1 0  0 1  000    1  01    0    0   100  1 10101 1 00   001 0 0  0  1 001  0  0010 0    1    0    101   00010   00     0  0 1 11 1 101  00  10 1 0 01 010  11   0  0 1     1 0  0  01  00 1 1001 0 1  10 01 010 101 0 0   10   01111   0100 111  011 11 0 00 010     10100     0   1 00 011  1 0  1010111   1 00111 000  00101  0 0 01 1 0 0  0 1 1   0   011   010100 11 1 0  1 1  1 1  1 1  01 1 0  00 000  0111  1 01 00  100010      00     00 1 0 0     10   01  0 1 0  0 1 1 01   11    10    0 1 0    1  0 1    1    0  0110 1101

# Compare Alice and Bob's sifted keys.


In [23]:
numMatch = 0
if len(siftedAlice) != len(siftedBob):
    print("Sifted keys are different lengths!")
else:
    for i in range(len(siftedAlice)):
        if siftedAlice[i] == siftedBob[i]:
           numMatch += 1
    matchPercent = numMatch / len(siftedAlice) * 100
print(str(matchPercent) + "% match")

68.9423076923077% match


# New to this implementation - Alice and Bob select a portion of their sifted keys to sample.

In [24]:

# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
sampleIndex = ""

fracSampled = 0.1
numSampled = 0

for i in range(len(siftedAlice)):
    if siftedAlice[i] != ' ' and random.uniform(0, 1) < fracSampled:
        sampleIndex += "1"
        numSampled += 1
    else:
        sampleIndex += "0"
print("sampleIndex = " + sampleIndex)

sampleIndex = 00000000000000000000000000000000000000000000000000000000000001001000000000000000000000000000000000000000000000000000000001000000100000000001000010000000000001000000000000000001000000000000000000000100000000001010001000000000000010000000000000000000000000000000010000000000000000100010000000000000000000000001000000000000001000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000010000000000000000000000000000000000001000000000000000001000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000010000000000000000000000000000000000001000000000000000000010000000000000000000000000000000000010000000000000000000000000000000000000001000000010000000000000010000000100000000000000000000000000000001000100010000000000000000000000000000100000000000000000010100000000000000000000000000001000000000010000100000000010000000000000000000000000000000000000000000000000000000000

In [25]:
# Alice and Bob use their sample to estimate the quantum bit error rate (QBER).
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampledBobQBER = 0
numSampled = 0
numMatch = 0
for i in range(len(sampleIndex)):
    if sampleIndex[i] == '1':
        numSampled += 1
    if (sampleIndex[i] == '1') and (siftedAlice[i] == siftedBob[i]):
        numMatch += 1
if numSampled > 0:
    sampledBobQBER = 1 - numMatch / numSampled
else:
    sampledBobQBER = float('nan')
    
    

In [26]:
#len(sampleIndex)
print(len(siftedBob))
print(len(sampleIndex))

print( "siftedAlice " + siftedAlice)
print("siftedBob " + siftedBob)

1040
1040
siftedAlice 111  0 0 11111 0 11010001  00        01 0 00 0 1 10  01 10 0 1 011000110 10  1 1   1 0 00 0 1 01       0   0  1   01     1 0   1011  01101 001  01  1 01000  0    11  10000    1 1 011110 00   1101  1   1  0 1 01001 0 1 1  1  0   11000 010 1 110 1 1  01 11100    1    0    0   10 0 01111  110  1     10   1 1 1    0 1  10   1 0 000  01 0 1100 1 10 10  1 1  1 0 01 011 10111011  0001 10   1 0   10001        0 101  0 1 00 0 0         01 0    0  10101 011 00 1  01 1 1   1 0  0 1  000    1  01    0    0   100  1 10101 1 00   001 0 0  0  1 001  0  0010 0    1    0    101   00010   00     0  0 1 11 1 101  00  10 1 0 01 010  11   0  0 1     1 0  0  01  00 1 1001 0 1  10 01 010 101 0 0   10   01111   0100 111  011 11 0 00 010     10100     0   1 00 011  1 0  1010111   1 00111 000  00101  0 0 01 1 0 0  0 1 1   0   011   010100 11 1 0  1 1  1 1  1 1  01 1 0  00 000  0111  1 01 00  100010      00     00 1 0 0     10   01  0 1 0  0 1 1 01   11    10    0 1 0    1  0 1    1    0  0

In [27]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""

secure_Bob_count = 0

for i in range(len(sampleIndex)):
    if (sampleIndex[i] == '1'):
        secureAlice += " "
    elif (sampleIndex[i] == '0') and siftedAlice[i] == '0':
        secureAlice += '0'
    elif (sampleIndex[i] == '0') and siftedAlice[i] == '1':
        secureAlice += '1'
    elif siftedAlice[i] == ' ':
        secureAlice += " "

print(secureAlice)
print(len(secureAlice))


for i in range(len(sampleIndex)):
    if (sampleIndex[i] == '1'):
        secureBob += " "
    elif (sampleIndex[i] == '0') and siftedBob[i] == '0':
        secureBob += '0'

    elif (sampleIndex[i] == '0') and siftedBob[i] == '1':
        secureBob += '1'

    elif siftedBob[i] == ' ':
        
        secureBob += " "
    else:
        secureBob += " "
                

111  0 0 11111 0 11010001  00        01 0 00 0 1 10  01 10 0   0 1000110 10  1 1   1 0 00 0 1 01       0   0  1   01       0   1 11  01101  01   1  1 01000       11  10000      1 011110 00   1101      1  0 1  1 01   1 1  1  0    1000 010 1 110 1 1  01 11100         0    0   10   01 11  110  1     10   1 1      0 1  10     0  00  01 0 1100 1 10 10  1 1  1 0 01 011 10111011  0001 10   1 0   10001        0 101  0 1 00 0           0  0    0  10101 011 00 1  01 1 1   1    0 1  000    1   1    0    0   100  1 10101 1 00   00  0 0  0  1 001  0  0010 0    1    0    101   00010   00     0  0 1 11 1 101  00   0 1 0 01 01   11   0  0 1     1 0  0  01  00 1 1 01 0 1  10 01 010 1 1 0 0   10   01111   0100 111  011  1 0 00 010     10100     0   1 00 011    0  101 111   1 00111  00  001 1  0 0 01 1 0 0  0 1 1   0   01     101 0 11 1 0  1 1  1 1  1 1  01   0  00 000  0111     1 00  100010      00     00   0 0     1    0   0 1 0    1 1 01   11    10    0 1 0    1  0 1    1    0  0110 11010  1  0  010 1

In [28]:
print("SI     " + sampleIndex[0:10])
print("SB     " + siftedBob[0:10])
print("secBob " +  secureBob[0:10])

SI     0000000000
SB     --1  - - -
secBob   1       


In [29]:
# Alice and Bob make a hard determination whether the channel is secure.
# TODO: Choose a good safety threshold.
safetyThreshold = 0.15  # change this value!

if sampledBobQBER > safetyThreshold:
    channelSecure = False  # something's fishy - hit the kill switch!
else:
    channelSecure = True  # everything looks fine!

print(channelSecure)

False


# Eve ------------------------------------------------------------------


In [30]:
# Below is the method Eve uses to extract her stolen key.
# You need not change any of what follows.

# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured
keyEve = ""
for i in range(len(interceptIndex)):
    if interceptIndex[i] == '1':
        if outcomeEve[i] == 'H' or outcomeEve[i] == 'D':
            keyEve += '0'
        elif outcomeEve[i] == 'V' or outcomeEve[i] == 'A':
            keyEve += '1'
        else:
            keyEve += '-'  # invalid measurement
    else:
        keyEve += ' '
        
print (keyEve)        


 - 1 1   1  1                      -        -    - 0               0        1 0 0 1                1 -             1         -               -      1                         -           -            -                -   -                -       1        -     1            - - 1  -          0          -       -         0      - --    -  -        01    --                   -   0        0       1           1-                -          -          --          -            -         --  -   -           -                                             1                                      -              1-           0            1 1  0   -                    1-      -    1                        0  1      -           - 0               -    1    -  -  - 1                              -         -      - -  -                     1-                      0             --                     -      -            1 0           0               0                            1                  -           

In [31]:
print(len(interceptIndex))
print(len(keyEve))
print(len(rawKeyBob))
print(len(basisAlice))
print(len(basisBob))
print(len(basisEve))

1040
1040
1040
1040
1040
1040


In [32]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publicly revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the convention '0', '1', ' '=removed
stolenEve = ""
for i in range(len(interceptIndex)):
    if interceptIndex[i] == '1' and keyEve[i] != '-' and rawKeyBob[i] != '-' and basisEve[i] == basisAlice[i] and basisEve[
        i] == basisBob[i]:
        stolenEve += keyEve[i]
    else:
        stolenEve += ' '
        
print(stolenEve)        

# ANALYSIS -------------------------------------------------------------


In [33]:
print(len(secureAlice))
print(len(secureBob))

print(secureAlice)
print(secureBob)

1040
1040
111  0 0 11111 0 11010001  00        01 0 00 0 1 10  01 10 0   0 1000110 10  1 1   1 0 00 0 1 01       0   0  1   01       0   1 11  01101  01   1  1 01000       11  10000      1 011110 00   1101      1  0 1  1 01   1 1  1  0    1000 010 1 110 1 1  01 11100         0    0   10   01 11  110  1     10   1 1      0 1  10     0  00  01 0 1100 1 10 10  1 1  1 0 01 011 10111011  0001 10   1 0   10001        0 101  0 1 00 0           0  0    0  10101 011 00 1  01 1 1   1    0 1  000    1   1    0    0   100  1 10101 1 00   00  0 0  0  1 001  0  0010 0    1    0    101   00010   00     0  0 1 11 1 101  00   0 1 0 01 01   11   0  0 1     1 0  0  01  00 1 1 01 0 1  10 01 010 1 1 0 0   10   01111   0100 111  011  1 0 00 010     10100     0   1 00 011    0  101 111   1 00111  00  001 1  0 0 01 1 0 0  0 1 1   0   01     101 0 11 1 0  1 1  1 1  1 1  01   0  00 000  0111     1 00  100010      00     00   0 0     1    0   0 1 0    1 1 01   11    10    0 1 0    1  0 1    1    0  0110 11010  1

In [34]:
# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
        secureKeyLengthBob += 1
        if secureAlice[i] == secureBob[i]:
            numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1 - actualBobQBER) * secureKeyLengthBob / n * 1e6;

In [35]:
# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1 - actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
        stolenKeyLengthEve += 1
        if secureAlice[i] == stolenEve[i]:
            numMatchEve += 1

In [36]:
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1 - actualEveQBER) * stolenKeyLengthEve / n * 1e6;


# DISPLAY RESULTS ------------------------------------------------------

In [37]:
print("")
print("rawKeyAlice     = " + rawKeyAlice)
print("basisAlice   = " + basisAlice)
print("photonAlice  = " + photonAlice)
print("basisEve     = " + basisEve)
print("outcomeEve   = " + outcomeEve)
print("keyEve       = " + keyEve)
print("basisBob     = " + basisBob)
print("outcomeBob   = " + outcomeBob)
print("rawKeyBob       = " + rawKeyBob)
print("")
print("siftedAlice  = " + siftedAlice)
print("siftedBob    = " + siftedBob)
print("")
print("secureAlice  = " + secureAlice)
print("secureBob    = " + secureBob)
print("stolenEve    = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob / 1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve / 1000) + " kbps")



rawKeyAlice     = 111110101111110011101000101001100011101000001011010000111010010011000110110111010011100000001001101111100110001010011001110000010111001101100101011111010000001010111010000000111100111100000011101001001110001101001001101111110111110001010111110011101011111001000100110110001111000101111111101110000010001111110111011111010010000000001100110001010110111111011000110111101110110100011101011000111000110011110001011000100010101110111110110110000010101101100001100111110011001000100000010010001111100010001010010101010111000011001100000010110010000100100011001000100101101000000100010010010011011111010101110010100110101001010110100010110010010010011010100011100110110010101001001011000000101000111101101000111000111111010000100010110100000100000110000110010010101011111010001110000110010100000001111010100111100101000110100101000110110001111111110111000111000100100010011111110110010100010011111000011100110010101011010001100111011011010010011100011010100011011011110011011101010110110

# Now that Alice and Bob have a key, they can send an encrypted message over public channels....

Alice now uses the one time pad encryption and the secret key to encrypt a message to send to Bob. (Bob will take the sifted key and apply it to binary-to-message string decoding) This will require some string processing in order to get the sifted keys into a format that we are used to dealing with.

In [38]:

# Remove whitespaces

import string


secure_stripped_key_Alice = secureAlice.translate({ord(c): None for c in string.whitespace})


secure_stripped_key_Bob =  secureBob.translate({ord(c): None for c in string.whitespace})




# This adds spaces every 8 binary digits tso that it can be used in the functions we have previously written.
final_secure_Alice = ''

for i in range(0,len(secure_stripped_key_Alice),8):
    final_secure_Alice += secure_stripped_key_Alice[i:i+8]
    final_secure_Alice += ' '

print( "secureAlice in groups of 8 : " + final_secure_Alice )
    
final_secure_Bob = ''

for i in range(0,len(secure_stripped_key_Bob),8):
    final_secure_Bob += secure_stripped_key_Bob[i:i+8]
    final_secure_Bob += ' '    
print( "secureBob in groups of 8 : " + final_secure_Bob )    
    
# chop off numbers at the end, so that the number of 0's and 1's are in multiples of 8

final_secure_Bob = final_secure_Bob[0: len(final_secure_Bob) - len(final_secure_Bob)%9]  
print( "secureBob in groups of 8 w/ end cut off : " + final_secure_Bob )    

final_secure_Alice = final_secure_Alice[0: len(final_secure_Alice) - len(final_secure_Alice)%9]   
print( "secureAlice in groups of 8 w/ end cut off : " + final_secure_Alice )    

    

secureAlice in groups of 8 : 11100111 11011010 00100010 00011001 10001000 11010111 00001010 01010111 01101011 10100011 10000101 11100011 01101101 11101000 01011101 10111100 00100111 11011011 01100000 10110011 01011100 10111011 10110001 10101000 10101010 00000101 01011001 01111010 00110010 01101011 00000001 00100010 01010100 01000001 11110100 01001011 10011000 10011010 11001010 11001001 11101001 11011100 00101010 00100011 01011111 00111000 01100011 00011001 10101110 11111101 00000001 11100100 01000000 01001011 01111001 01011001 10110101 00101111 00101011 00110111 0000 
secureBob in groups of 8 : 11111001 00000110 00111000 11000111 10000001 11001011 00011011 01100011 00001101 11011010 01001001 00110100 11010001 01100100 11000111 01011101 11101001 01111111 00100011 00111011 00010010 11111110 00011110 01100100 000110 
secureBob in groups of 8 w/ end cut off : 11111001 00000110 00111000 11000111 10000001 11001011 00011011 01100011 00001101 11011010 01001001 00110100 11010001 01100100 110001

# Alice encodes her message to binary 
These functions will be used in this portion, and are imported from previous assignments.

In [ ]:
def encode_to_decimal(sentence):
    result = []  ##empty list to store results
    
    for i in range( len(sentence)):
      
    #[ENTER CODE]
        # append result to list. remember that you use list_name[i]
        # to iterate through a list. Use the ord function on each list 
        #element
        
        
        result.append(ord(sentence[i]))
        
    print("result is" + str(result) )
    
    return result

def decimal_to_binary_string(number_list):
    
    binary_string = ""
    
    for i in range(len(number_list)):
        
        bin_number_whole = bin(number_list[i])
        
        #chop the  "Ob" off of the binary conversion string output
        
        bin_number_chopped = bin_number_whole[2: len(bin_number_whole)]
        
        bin_number_chopped = "0" + bin_number_chopped
        
        
        # To deal with the 'Space' unicode (32) we need to add a '0' at the head of the binary number to make it '00100000'
        
        if (bin_number_chopped == '0100000'):
            bin_number_chopped = '0'+ bin_number_chopped
            
        #[ENTER CODE]   you should have a binary number in 8 bit form. now you need to append it to a string, but add a space at the tail 
        
        binary_string += bin_number_chopped + ' '   
        
        print("binary number " + str(i) + " is " + bin_number_chopped)
        
    print("binary string is " + binary_string)
    return binary_string
        
        #iterate through list of numbers, converting each number to binary and appending
       # it to a string. You should end up with a long string of 0's and 1's
    
def one_time_encrypt(original_binary_message, key):
    
    if(len(original_binary_message)< len(key)):
        
        difference = len(original_binary_message) - len(key)
        
        print( "Key is longer than message, don't need " + str(abs(difference)) + " key digits ")
        
        key = key[0: (len(key) - difference)]
        
    if (len(original_binary_message)> len(key)):
        raise Exception( " Adjust number of photons and start over, you don't have enough key to send this")
    
   # print(chopped_secret_key)
   # print(len(chopped_secret_key))
    
    encrypted_binary_string = ''
    
    print( "key is " + key)
    
    for i in range(len(original_binary_message)):
     
      #[ENTER CODE] Apply mod 2 sum logic
                   
         if ((original_binary_message[i] == '0') and (key[i] == '0')):
            encrypted_binary_string += '0'
            
         if ((original_binary_message[i] == '0') and (key[i] == '1')):
            encrypted_binary_string += '1'  
            
         if ((original_binary_message[i] == '1') and (key[i] == '0')):
            encrypted_binary_string += '1'  
            
         if ((original_binary_message[i] == '1') and (key[i] == '1')):
            encrypted_binary_string += '0' 
        
         if ((original_binary_message[i] == ' ') and (key[i] == ' ')):
            encrypted_binary_string += ' '  
      
         #print ( "encrypted string so far " + encrypted_binary_string )
            
    return  encrypted_binary_string  
                  
def apply_key( encrypted_binary_string, key):
    
    unencrypted_binary_string = ''
        
    print( "encrypted_binary_string " + encrypted_binary_string)
    print("key is " + key)
    
    if len(encrypted_binary_string) > len(key):
        raise Exception(" Bob does not have enough key to decrypt, try sending more photons")

    
    for i in range(len(encrypted_binary_string)):
    
        # [ENTER CODE] Apply mod 2 sum logic

        if ((encrypted_binary_string[i] == '0') and (key[i] == '0')):
                   unencrypted_binary_string  += '0'

        if ((encrypted_binary_string[i] == '0') and (key[i] == '1')):
                   unencrypted_binary_string  += '1'  

        if ((encrypted_binary_string[i] == '1') and (key[i] == '0')):
                   unencrypted_binary_string  += '1'  

        if ((encrypted_binary_string[i] == '1') and (key[i] == '1')):
                     unencrypted_binary_string += '0'
        
        if ((encrypted_binary_string[i] == ' ') and (key[i] == ' ')):
                     unencrypted_binary_string += ' '
    
    return  unencrypted_binary_string  


def decode(binary_string):
    
    end_string = '' # create and empty string
    
    for i in range (0,len(binary_string), 9): 
        
       my_binary = binary_string[i : i+9]
       
       print("my_binary is " + my_binary)
    
       my_int = int(my_binary,2)
        
       print( "my_int is " + str(my_int)) 
        
       my_character = chr(my_int) 
        
       print("my_character is " + my_character)
    
    
       # [ENTER CODE] append the decoded character my_character to end_string
 
    
       end_string += my_character
        
       
       # Remember that you can't use the append function here. Why? What
        # alternative method can you use?
    print(end_string)    
    return end_string       


In [ ]:

# convert Alice's message to decimal

am_decimal = encode_to_decimal(alice_message)

# decimal to binary string

am_binary_string = decimal_to_binary_string(am_decimal)

# Use Alice's sifted key to one-time encrypt

encrypted_message = one_time_encrypt( am_binary_string,final_secure_Alice)


# Alice can now send encrypted_message over a public channel

    
 


# Bob receives encrypted message from Alice, uses his sifted key to decrypt




In [ ]:
bob_final_binary = apply_key(encrypted_message, final_secure_Bob)

bob_final_message = decode(bob_final_binary)

In [ ]:
print(bob_final_binary)

In [ ]:
#In this implementation, these messages should be the same. Once we introduce Eve,
# there may be some problems.

bob_final_message